# Biostat823 Assignment3

Biostat823 assignent3 Ashley Hu

**3**. Creating effective visualizations using [best practices](https://rafalab.github.io/dsbook/data-visualization-principles.html)

Create 3 informative visualizations about malaria using Python in a Jupyter notebook, starting with the data sets at https://github.com/rfordatascience/tidytuesday/tree/master/data/2018/2018-11-13. Where appropriate, make the visualizations [interactive](https://jupyterbook.org/interactive/interactive.html).

**Note** There are many libraries you can use for each task. Choose one library and explain why you chose it in your blog.

This assignment use three dataset namely **malaria_deaths.csv**, **malaria_deaths_age.csv**, **malaria_inc.scv** that give information on death rate for all age group(between 1990 and 2016), the number of death cases grouped by age (between 1990 and 2016) and the incidence of malaria (between 2000 and 2015). I used the plotting library **bokeh** to generate plot1 and plot2. I used the plotting library **matplotlib** to generate plot3.

In [361]:
import csv
import numpy as np
import pandas as pd

In [362]:
pip install pandas-bokeh

Note: you may need to restart the kernel to use updated packages.


In [377]:
import pandas_bokeh
from bokeh.io import output_notebook, curdoc
from bokeh.plotting import figure, show, output_notebook, output_file
from bokeh.layouts import gridplot, row, column
from bokeh.models import ColumnDataSource, DataRange1d
from bokeh.models.tools import HoverTool
from bokeh.transform import factor_cmap
from bokeh.palettes import brewer
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
import ipywidgets as widgets
output_notebook()

Loading BokehJS ...

In [364]:
#Read the malaria_deaths.csv file
death = pd.read_csv('malaria_deaths.csv')
death = death.rename(columns={"Deaths - Malaria - Sex: Both - Age: Age-standardized (Rate) (per 100,000 people)":"death_rate"})
# To print the first 5 rows of the malaria_deaths.csv file
death.head()

,Entity,Code,Year,death_rate
0,Afghanistan,AFG,1990,6.802930
1,Afghanistan,AFG,1991,6.973494
2,Afghanistan,AFG,1992,6.989882
3,Afghanistan,AFG,1993,7.088983
4,Afghanistan,AFG,1994,7.392472


In [365]:
#Read the malaria_deaths_age.csv
deathage = pd.read_csv('malaria_deaths_age.csv')
deathage = deathage.rename(columns = {'age_group':'agegroup'})
#To print the first 5 rows of the malaria_deaths_age.csv
deathage.head()

,Unnamed: 0,entity,code,year,agegroup,deaths
0,1,Afghanistan,AFG,1990,Under 5,184.606435
1,2,Afghanistan,AFG,1991,Under 5,191.658193
2,3,Afghanistan,AFG,1992,Under 5,197.140197
3,4,Afghanistan,AFG,1993,Under 5,207.357753
4,5,Afghanistan,AFG,1994,Under 5,226.209363


In [366]:
#Read the malaria_inc.csv file
inc = pd.read_csv('malaria_inc.csv')
inc = inc.rename(columns = {"Incidence of malaria (per 1,000 population at risk) (per 1,000 population at risk)":"incidence"})
#To print the first 5 rows of the malaria_inc.csv
inc.head()

,Entity,Code,Year,incidence
0,Afghanistan,AFG,2000,107.100000
1,Afghanistan,AFG,2005,46.500000
2,Afghanistan,AFG,2010,23.900000
3,Afghanistan,AFG,2015,23.600000
4,Algeria,DZA,2000,0.037746


### Plot1

I use **bokeh** to draw two interactive line plots to represent the incidence and overall death rate of malaria. The **bokeh** library offers hovertools to show the data at each datapoint by pointing at each dot. It also offer a tool bar where I can zoom-in and zoom-out the plot.

In [378]:
def get_incidence_plot(Country):
    """
    Country is the input, the input is the name of a country.
    eg. get_incidence_plot('China')
    >>> The output will be a column of two plots.
    """
    output_file("line_incidence.html")
    tooltips1 = [
            ('Incidence', '@incidence'),
            ('Year', '@Year')  
           ]
    df1 = inc.loc[inc['Entity']==Country]
    source1 = ColumnDataSource(df1)
    p1 = figure(title = 'Incidence of Malaria (per 1,000 people at risk)',
              width = 600, height = 400)
    p1.line(x = 'Year',
           y = 'incidence',
           source = source1)
    p1.scatter(x = 'Year',
              y = 'incidence',
             source = source1)
    p1.xaxis.axis_label = 'Year'
    p1.yaxis.axis_label = 'Incidence (per 1,000 people at risk)'
    p1.add_tools(HoverTool(tooltips = tooltips1))
    tooltips2 = [
             ('Death rate', '@death_rate'),
             ('Year', '@Year')
    ]
    df2 = death.loc[death['Entity']==Country]
    source2 = ColumnDataSource(df2)
    p2 = figure(title = 'Death rate of Malaria all age group (per 100,000 people)',
               width = 600, height = 400)
    p2.line(x = 'Year',
           y = 'death_rate',
           source = source2)
    p2.scatter(x = 'Year',
              y = 'death_rate',
              source = source2)
    p2.xaxis.axis_label = 'Year'
    p2.yaxis.axis_label = 'Death rate (per 100,000 people)'
    p2.add_tools(HoverTool(tooltips = tooltips2))
    return show(column(p1,p2))

In [379]:
#Use the widgets to select country names.
widgets.interact(get_incidence_plot, Country = inc["Entity"].unique())

interactive(children=(Dropdown(description='Country', options=('Afghanistan', 'Algeria', 'Angola', 'Argentina'…

<function __main__.get_incidence_plot(Country)>

### Plot 2

I used the **bokeh** library to plot a stacked line plot to represent the number of deaths by year grouped by age.

In [375]:
def get_age_death_plot(Country):
    #Get data 
    df1 = deathage.loc[(deathage['entity']==Country)]
    df2 = df1.pivot(index = 'year', columns = 'agegroup', values = 'deaths')
    data = dict(
        years = list(deathage['year'].unique()),
        g1 = list(df2['Under 5']),
        g2 = list(df2['14-May']),
        g3 = list(df2['15-49']),
        g4 = list(df2['50-69']),
        g5 = list(df2['70 or older']))
    source = ColumnDataSource(data)
    #Draw the plot
    output_file("area_agedeath_grouped.html")
    p = figure(width = 700, height = 400, 
           title = 'Death cases by age group',
           x_axis_label = 'Year',
           y_axis_label = 'Death case')
    p.varea_stack(['g1', 'g2', 'g3', 'g4', 'g5'], x='years', 
              color = ('lightsalmon', 'salmon','indianred','crimson','firebrick'), 
              legend_label = ('<5', '6-14', '15-49', '50-69', '>70'),
              source = source,
              fill_alpha = 0.9)
    p.legend.title = 'Age'
    p.legend.location = 'top_left'
    return show(p)

In [376]:
widgets.interact(get_age_death_plot, Country = deathage["entity"].unique())

interactive(children=(Dropdown(description='Country', options=('Afghanistan', 'Albania', 'Algeria', 'American …

<function __main__.get_age_death_plot(Country)>

### Plot3

I used the **matplotlib** library to generate a map to show the incidence of malaria between 2000 and 2015. I used the naturalearth shapefile to plot the country shape and use the incidence data to fill in the plot.

In [380]:
import geopandas as gpd
import matplotlib.pyplot as plt

In [381]:
#Get world map data for plotting
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [382]:
def get_incidence_geographical_plot(year):
    df = inc.loc[inc['Year']==year]
    key = df['Entity']
    gdf = world.merge(df, left_on = world['name'], right_on = df['Entity'], how = 'outer')
    fig, ax = plt.subplots(1, figsize=(12,5))

    gdf.plot(column='incidence',  cmap='OrRd', ax=ax, legend=True,
        legend_kwds={'label': "Incidence of malaria (per 1,000 population at risk)",
                     'orientation': "horizontal"})

In [374]:
widgets.interact(get_incidence_geographical_plot, year = inc['Year'].unique())

interactive(children=(Dropdown(description='year', options=(2000, 2005, 2010, 2015), value=2000), Output()), _…

<function __main__.get_incidence_geographical_plot(year)>